In [39]:
import os
import astropy.io.fits as fits
import datetime as dt
import gzip
import re
import glob
from pprint import pprint as pp

import requests
import json

In [40]:
root_dir = "/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test"
ALLOWED_EXT = ['.gz', '.bz2']
forbidden_keys = ['HISTORY', 'COMMENT']

In [52]:
def get_dirs_to_walk(root_dir):
    folders_in_root = os.listdir(root_dir)
    p = re.compile('^\d{4}-\d{2}-\d{2}$')
    
    dirs_to_walk = []
    for _d in os.listdir(root_dir):
        if p.match(_d):
            dirs_to_walk.append(os.path.join(root_dir, _d))
    return dirs_to_walk


def get_files(_dir):
    
    files_in_folder = []
    for r, d, f in os.walk(_dir):
        for file in f:
            files_in_folder.append(os.path.join(r, file))
    
    files_to_open = []
    for _file in files_in_folder:
        if os.path.isfile(_file):
            if os.path.splitext(_file)[-1] in ALLOWED_EXT:
                files_to_open.append(_file)
    
    return files_to_open


def get_folder_data(files_to_open):
    folder_data = []
    for f in files_to_open:
        with gzip.open(f, 'rb') as f_in:
            hdr = dict(fits.getheader(f))
            
            obs_datetime = dt.datetime.strptime(
                hdr['DATE-OBS'] + 'T' + hdr['TIME-OBS'],
                '%Y-%m-%dT%H:%M:%S.%f').isoformat()
            
            object_name = hdr['OBJECT']
            observer = hdr['OBSERVER']
            color_filter = hdr['FILTER']
            exptime = hdr['EXPTIME']
            
        row = {
            'obs_datetime': obs_datetime,
            'object_name': object_name,
            'observer': observer,
            'color_filter': color_filter,
            'exptime': exptime,
        }
        folder_data.append(row)
        
    folder_data = sorted(folder_data, key=lambda x: x['obs_datetime'])
    
    return folder_data


def get_grouped_folder_data(folder_data):
    results = []
    target = {}
    n_frames = 0
    prev_name = None

    for frame in folder_data:
        if frame['object_name'] != prev_name:
            n_frames = 0
            if target:
                target['datetime_end'] = frame['obs_datetime']
                target['colorfilters'] = [
                    {'name': c} for c in target['colorfilters']
                ]
                results.append(target)
            target = {}
            prev_name = frame['object_name']
            target['name'] = frame['object_name']
            target['datetime_start'] = frame['obs_datetime']
            target['observer'] = frame['observer']
            target['colorfilters'] = []
            target['total_exposure_time'] = 0
            target['number_of_frames'] = 0
            target['telescope'] = 'Zeiss60'

        target['number_of_frames'] += 1    
        target['total_exposure_time'] += float(frame.get('exptime', 0))
        if frame['color_filter'] not in target['colorfilters']:
            target['colorfilters'].append(frame['color_filter'])

    return results
    


In [53]:
dirs_to_walk = sorted(get_dirs_to_walk(root_dir))
data_to_send = []
for _dir in dirs_to_walk:
    print('#'*9)
    pp(_dir)
    folder_files = get_files(_dir)
    folder_data = get_folder_data(folder_files)
    grouped_folder_data = get_grouped_folder_data(folder_data)
    data_to_send.append(grouped_folder_data)
    pp(grouped_folder_data)
    print('#'*9)

#########
'/home/pi/GoogleDrive/mydrive/Data/astro/suhora_logs/logs_test/2010-05-26'
[{'colorfilters': [{'name': 'U'}],
  'datetime_end': '2010-05-26T18:21:03.680000',
  'datetime_start': '2010-05-26T18:20:33.920000',
  'name': 'bias',
  'number_of_frames': 9,
  'observer': 'win',
  'telescope': 'Zeiss60',
  'total_exposure_time': 0.0},
 {'colorfilters': [{'name': 'U'}],
  'datetime_end': '2010-05-26T19:11:14.030000',
  'datetime_start': '2010-05-26T18:21:03.680000',
  'name': 'dark',
  'number_of_frames': 9,
  'observer': 'win',
  'telescope': 'Zeiss60',
  'total_exposure_time': 540.0},
 {'colorfilters': [{'name': 'B'}, {'name': 'I'}, {'name': 'R'}, {'name': 'V'}],
  'datetime_end': '2010-05-26T20:21:02.420000',
  'datetime_start': '2010-05-26T19:11:14.030000',
  'name': 'flat',
  'number_of_frames': 40,
  'observer': 'win',
  'telescope': 'Zeiss60',
  'total_exposure_time': 200.0},
 {'colorfilters': [{'name': 'R'}],
  'datetime_end': '2010-05-26T22:16:00.450000',
  'datetime_start': 

[{'colorfilters': [{'name': 'R'}],
  'datetime_end': '2021-01-28T04:42:59.140000',
  'datetime_start': '2021-01-28T04:17:25.150000',
  'name': 'oj287',
  'number_of_frames': 17,
  'observer': 'MD',
  'telescope': 'Zeiss60',
  'total_exposure_time': 1020.0},
 {'colorfilters': [{'name': 'R'}],
  'datetime_end': '2021-01-28T05:09:10.790000',
  'datetime_start': '2021-01-28T04:42:59.140000',
  'name': 'j1723',
  'number_of_frames': 16,
  'observer': 'MD',
  'telescope': 'Zeiss60',
  'total_exposure_time': 960.0},
 {'colorfilters': [{'name': 'R'}],
  'datetime_end': '2021-01-28T05:40:17.250000',
  'datetime_start': '2021-01-28T05:09:10.790000',
  'name': 'j2042',
  'number_of_frames': 15,
  'observer': 'MD',
  'telescope': 'Zeiss60',
  'total_exposure_time': 900.0},
 {'colorfilters': [{'name': 'B'}, {'name': 'V'}, {'name': 'I'}, {'name': 'R'}],
  'datetime_end': '2021-01-28T06:06:21.680000',
  'datetime_start': '2021-01-28T05:40:17.250000',
  'name': 'flat',
  'number_of_frames': 36,
  'obs

In [67]:
for folder_ds in data_to_send:
    for ds in folder_ds:
        response = requests.post('http://127.0.0.1:8000/targets/', auth=('admin', 'haslobrzmi'),
                  data=json.dumps(ds),
                  headers={'content-type': 'application/json'})

        print(response)
        print(response.content)

<Response [400]>
b'{"non_field_errors":["The fields datetime_start, telescope must make a unique set."]}'
<Response [400]>
b'{"non_field_errors":["The fields datetime_start, telescope must make a unique set."]}'
<Response [400]>
b'{"non_field_errors":["The fields datetime_start, telescope must make a unique set."]}'
<Response [400]>
b'{"non_field_errors":["The fields datetime_start, telescope must make a unique set."]}'
<Response [201]>
b'{"datetime_start":"2015-10-10T20:47:21.400000Z","datetime_end":"2015-10-10T21:23:51.130000Z","observer":"siwak","name":"lkha","ra":null,"dec":null,"note":"","telescope":"Zeiss60","colorfilters":[{"name":"GS"},{"name":"IS"}],"total_exposure_time":"1000.00","number_of_frames":40}'
<Response [201]>
b'{"datetime_start":"2015-10-10T21:23:51.130000Z","datetime_end":"2015-10-10T22:03:39.840000Z","observer":"siwak","name":"ngc7129","ra":null,"dec":null,"note":"","telescope":"Zeiss60","colorfilters":[{"name":"GS"},{"name":"IS"}],"total_exposure_time":"1830.00"

<Response [201]>
b'{"datetime_start":"2021-01-28T05:40:17.250000Z","datetime_end":"2021-01-28T06:06:21.680000Z","observer":"MD","name":"flat","ra":null,"dec":null,"note":"","telescope":"Zeiss60","colorfilters":[{"name":"B"},{"name":"V"},{"name":"I"},{"name":"R"}],"total_exposure_time":"180.00","number_of_frames":36}'
<Response [201]>
b'{"datetime_start":"2021-01-28T06:06:21.680000Z","datetime_end":"2021-01-28T06:07:34.620000Z","observer":"MD","name":"bias","ra":null,"dec":null,"note":"","telescope":"Zeiss60","colorfilters":[{"name":"B"}],"total_exposure_time":"0.00","number_of_frames":50}'


In [56]:
response = requests.post('http://127.0.0.1:8000/targets/', auth=('admin', 'haslobrzmi'),
              data=json.dumps(data_to_send[0][0]),
              headers={'content-type': 'application/json'})

print(response)
print(response.content)

<Response [400]>
b'{"non_field_errors":["The fields datetime_start, telescope must make a unique set."]}'


In [59]:
ds

[{'name': 'oj287',
  'datetime_start': '2021-01-28T04:17:25.150000',
  'observer': 'MD',
  'colorfilters': [{'name': 'R'}],
  'total_exposure_time': 1020.0,
  'number_of_frames': 17,
  'telescope': 'Zeiss60',
  'datetime_end': '2021-01-28T04:42:59.140000'},
 {'name': 'j1723',
  'datetime_start': '2021-01-28T04:42:59.140000',
  'observer': 'MD',
  'colorfilters': [{'name': 'R'}],
  'total_exposure_time': 960.0,
  'number_of_frames': 16,
  'telescope': 'Zeiss60',
  'datetime_end': '2021-01-28T05:09:10.790000'},
 {'name': 'j2042',
  'datetime_start': '2021-01-28T05:09:10.790000',
  'observer': 'MD',
  'colorfilters': [{'name': 'R'}],
  'total_exposure_time': 900.0,
  'number_of_frames': 15,
  'telescope': 'Zeiss60',
  'datetime_end': '2021-01-28T05:40:17.250000'},
 {'name': 'flat',
  'datetime_start': '2021-01-28T05:40:17.250000',
  'observer': 'MD',
  'colorfilters': [{'name': 'B'}, {'name': 'V'}, {'name': 'I'}, {'name': 'R'}],
  'total_exposure_time': 180.0,
  'number_of_frames': 36,
  

In [51]:
a = []

[{'v': i} for i in a]

[]

In [66]:
f"{round(10.2):d}"

'10'